In [1]:
import QUANTAXIS as QA

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.0.47
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##`````##```##```````##```````##````##```````###````````##``````###```` 
  ##````````##```##````````##`````##`````

# 在这里我们演示一下 下单/交易/结算的整个流程

我们首先会建立一个账户类和一个回测类


In [2]:
 # 初始化一个account
Account=QA.QA_Account()

# 初始化一个回测类
B = QA.QA_BacktestBroker()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: QUANTAXIS 1.0.46 has changed the init_assets ==> init_cash, please pay attention to this change if you using init_cash to initial an account class,                
  


在第一天的时候,全仓买入 000001

In [3]:
# 全仓买入'000001'

Order=Account.send_order(code='000001',
                        price=11,
                        money=Account.cash_available,
                        time='2018-05-09',
                        towards=QA.ORDER_DIRECTION.BUY,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_MONEY
                        )




In [4]:

## 打印order的占用资金
print('ORDER的占用资金: {}'.format((Order.amount*Order.price)*(1+Account.commission_coeff)))

ORDER的占用资金: 999049.7000000001


In [5]:
# 账户剩余资金
print('账户剩余资金 :{}'.format(Account.cash_available))

账户剩余资金 :0


In [6]:
Account.hold

Series([], Name: amount, dtype: float64)

In [7]:
Account.init_hold

Series([], Name: amount, dtype: float64)

In [8]:
Account.hold_available

Series([], Name: amount, dtype: float64)

此时的账户cash并未减少,因为此过程为申报订单(已委托 未成交状态)

回测类接受订单,并返回撮合结果

In [9]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

{'header': {'source': 'market', 'status': 200, 'code': '000001', 'session': {'user': None, 'strategy': None, 'account': 'Acc_Nem730lP'}, 'order_id': 'Order_v8jXEC3B', 'trade_id': 'Trade_b5NdHMSO'}, 'body': {'order': {'price': 10.96, 'code': '000001', 'amount': 90800, 'date': '2018-05-09', 'datetime': '2018-05-09 09:30:00', 'towards': 1}, 'fee': {'commission': 248.67849999999999, 'tax': 0}}}


账户类接收到回测返回的回报信息,更新账户

In [10]:
Account.receive_deal(rec_mes)

{'source': 'account',
 'account_cookie': 'Acc_Nem730lP',
 'portfolio_cookie': None,
 'user_cookie': None,
 'broker': 'backtest',
 'market_type': 'stock_cn',
 'strategy_name': None,
 'current_time': None,
 'allow_sellopen': False,
 'allow_t0': False,
 'margin_level': False,
 'init_assets': {'cash': 1000000, 'hold': {}},
 'commission_coeff': 0.00025,
 'tax_coeff': 0.0015,
 'cash': [1000000, 4583.321499999845],
 'history': [['2018-05-09 09:30:00',
   '000001',
   10.96,
   90800.0,
   'Order_v8jXEC3B',
   'Trade_b5NdHMSO',
   'Acc_Nem730lP',
   248.67849999999999,
   0.0]],
 'trade_index': ['2018-05-09 09:30:00'],
 'running_time': datetime.datetime(2018, 6, 12, 21, 15, 16, 605963),
 'quantaxis_version': '1.0.47',
 'running_environment': 'backtest'}

此时我们可以打印一下现在的状态(现在的状态可以理解为在交易时 买入一只000001股票,但当天尚未收盘)

In [11]:
print('账户的可用资金 {}'.format(Account.cash_available))

账户的可用资金 4583.321499999845


In [12]:
Account.hold

code
000001    90800.0
Name: amount, dtype: float64

In [13]:
Account.hold_available

code
000001    90800.0
Name: amount, dtype: float64

In [14]:
Account.init_hold.index_name='code'

In [15]:
import pandas as pd

In [16]:
pd.concat([Account.hold_available,Account.init_hold])

code
000001    90800.0
Name: amount, dtype: float64

In [17]:
+Account.hold_available

code
000001    90800.0
Name: amount, dtype: float64

我们注意到 当最初申报订单的时候,可用资金只有950.2999999999302元,而买入成功后,可用资金有3339.9289999998837元,原因是下单的时候模式是市价单模式(QA.ORDER_MODEL.MARKET),故实际成交金额为10.96元

买入以后 账户的持仓为90800股 000001

In [18]:
Account.hold

code
000001    90800.0
Name: amount, dtype: float64

买入后账户现金表被扩展

In [19]:
Account.cash

[1000000, 4583.321499999845]

因为是t+1的A股市场,故此时可卖数量为0

In [20]:
Account.sell_available

Series([], Name: amount, dtype: float64)

# 执行结算

In [21]:
Account.settle()

# 结算后

In [22]:
Account.cash

[1000000, 4583.321499999845]

In [23]:
Account.cash_available

4583.321499999845

In [24]:
Account.sell_available

code
000001    90800.0
Name: amount, dtype: float64

In [25]:
Account.hold

code
000001    90800.0
Name: amount, dtype: float64

# 执行卖出操作

现在的持仓为: 000001 90800股

In [26]:
holdnum=Account.sell_available.get('000001',0)

In [27]:
holdnum

90800.0

申报一个卖出单,把可卖全部卖出

In [28]:
Order=Account.send_order(code='000001',
                        price=11,
                        amount=holdnum,
                        time='2018-05-10',
                        towards=QA.ORDER_DIRECTION.SELL,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_AMOUNT
                        )


In [29]:
Order

< QA_Order datetime:2018-05-10 09:31:00 code:000001 amount:90800.0 price:11 towards:-1 btype:stock_cn order_id:Order_643GpnKg account:Acc_Nem730lP status:300 >

In [30]:
Account.cash_available  # 因为此时订单尚未申报成功 可用现金不变

4583.321499999845

In [31]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

{'header': {'source': 'market', 'status': 200, 'code': '000001', 'session': {'user': None, 'strategy': None, 'account': 'Acc_Nem730lP'}, 'order_id': 'Order_643GpnKg', 'trade_id': 'Trade_BLhEHNtY'}, 'body': {'order': {'price': 11.0, 'code': '000001', 'amount': 90800.0, 'date': '2018-05-10', 'datetime': '2018-05-10 09:30:00', 'towards': -1}, 'fee': {'commission': 249.7, 'tax': 1498.2}}}


In [32]:
Account.receive_deal(rec_mes)

{'source': 'account',
 'account_cookie': 'Acc_Nem730lP',
 'portfolio_cookie': None,
 'user_cookie': None,
 'broker': 'backtest',
 'market_type': 'stock_cn',
 'strategy_name': None,
 'current_time': None,
 'allow_sellopen': False,
 'allow_t0': False,
 'margin_level': False,
 'init_assets': {'cash': 1000000, 'hold': {}},
 'commission_coeff': 0.00025,
 'tax_coeff': 0.0015,
 'cash': [1000000, 4583.321499999845, 1001635.4214999999],
 'history': [['2018-05-09 09:30:00',
   '000001',
   10.96,
   90800.0,
   'Order_v8jXEC3B',
   'Trade_b5NdHMSO',
   'Acc_Nem730lP',
   248.67849999999999,
   0.0],
  ['2018-05-10 09:30:00',
   '000001',
   11.0,
   -90800.0,
   'Order_643GpnKg',
   'Trade_BLhEHNtY',
   'Acc_Nem730lP',
   249.7,
   1498.2]],
 'trade_index': ['2018-05-09 09:30:00', '2018-05-10 09:30:00'],
 'running_time': datetime.datetime(2018, 6, 12, 21, 15, 16, 914971),
 'quantaxis_version': '1.0.47',
 'running_environment': 'backtest'}

In [33]:
Account.cash_available # 此时订单已成交 cash_available立刻结转

1001635.4214999999

In [34]:
Account.history_table

,datetime,code,price,amount,order_id,trade_id,account_cookie,commission,tax
0,2018-05-09 09:30:00,000001,10.96,90800.0,Order_v8jXEC3B,Trade_b5NdHMSO,Acc_Nem730lP,248.6785,0.0
1,2018-05-10 09:30:00,000001,11.00,-90800.0,Order_643GpnKg,Trade_BLhEHNtY,Acc_Nem730lP,249.7000,1498.2


In [35]:
Account.orders

< QA_OrderQueue AMOUNT 2 WAITING TRADE 2 >

In [36]:
Account.orders.queue_df.query('date=="2018-05-09"')

,account_cookie,amount,amount_model,callback,code,commission_coeff,date,datetime,frequence,market_type,...,price,sending_time,status,strategy,tax_coeff,towards,trade_id,transact_time,type,user
order_id,,,,,,,,,,,,,,,,,,,,,
Order_v8jXEC3B,Acc_Nem730lP,90800.0,by_money,<bound method QA_Account.receive_deal of < QA_...,000001,0.00025,2018-05-09,2018-05-09 09:31:00,day,stock_cn,...,11,2018-05-09 09:31:00,300,None,0.0015,1,None,None,stock_cn,None


# 测试T0账户

In [37]:
 # 初始化一个account
AccountT0=QA.QA_Account(running_environment=QA.RUNNING_ENVIRONMENT.TZERO,init_hold={'000001':10000},init_cash=200000)

# 初始化一个回测类
B = QA.QA_BacktestBroker()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: QUANTAXIS 1.0.46 has changed the init_assets ==> init_cash, please pay attention to this change if you using init_cash to initial an account class,                
  


In [38]:
AccountT0.init_assets

{'cash': 200000, 'hold': {'000001': 10000}}

In [39]:
AccountT0.init_hold

code
000001    10000
Name: amount, dtype: int64

In [40]:
AccountT0.hold_available

Series([], Name: amount, dtype: float64)

In [41]:
AccountT0.sell_available

code
000001    10000
Name: amount, dtype: int64

In [42]:
Order=AccountT0.send_order(code='000001',
                        price=11,
                        amount=AccountT0.sell_available.get('000001',0),
                        time='2018-05-10',
                        towards=QA.ORDER_DIRECTION.SELL,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_AMOUNT
                        )


In [43]:
Order

< QA_Order datetime:2018-05-10 09:31:00 code:000001 amount:10000 price:11 towards:-1 btype:stock_cn order_id:Order_XvZmzJ12 account:Acc_2JZYw3aV status:300 >

In [44]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

{'header': {'source': 'market', 'status': 200, 'code': '000001', 'session': {'user': None, 'strategy': None, 'account': 'Acc_2JZYw3aV'}, 'order_id': 'Order_XvZmzJ12', 'trade_id': 'Trade_6fs3dvFZ'}, 'body': {'order': {'price': 11.0, 'code': '000001', 'amount': 10000, 'date': '2018-05-10', 'datetime': '2018-05-10 09:30:00', 'towards': -1}, 'fee': {'commission': 27.5, 'tax': 165.0}}}


In [45]:
AccountT0.receive_deal(rec_mes)

{'source': 'account',
 'account_cookie': 'Acc_2JZYw3aV',
 'portfolio_cookie': None,
 'user_cookie': None,
 'broker': 'backtest',
 'market_type': 'stock_cn',
 'strategy_name': None,
 'current_time': None,
 'allow_sellopen': False,
 'allow_t0': False,
 'margin_level': False,
 'init_assets': {'cash': 200000, 'hold': {'000001': 10000}},
 'commission_coeff': 0.00025,
 'tax_coeff': 0.0015,
 'cash': [200000, 309807.5],
 'history': [['2018-05-10 09:30:00',
   '000001',
   11.0,
   -10000.0,
   'Order_XvZmzJ12',
   'Trade_6fs3dvFZ',
   'Acc_2JZYw3aV',
   27.5,
   165.0]],
 'trade_index': ['2018-05-10 09:30:00'],
 'running_time': datetime.datetime(2018, 6, 12, 21, 15, 17, 140154),
 'quantaxis_version': '1.0.47',
 'running_environment': 't0'}

In [46]:
AccountT0.sell_available

code
000001    0
Name: amount, dtype: int64

In [47]:
AccountT0.hold

code
000001    0.0
Name: amount, dtype: float64

In [48]:
AccountT0.buy_available

code
000001    10000
Name: amount, dtype: int64

In [49]:
AccountT0.hold_available

code
000001   -10000.0
Name: amount, dtype: float64

In [50]:
AccountT0.running_time

datetime.datetime(2018, 6, 12, 21, 15, 17, 17152)

In [51]:
AccountT0.datetime

'2018-05-10 09:30:00'

In [52]:
#AccountT0.close_positions_order

In [53]:
for item in AccountT0.close_positions_order:
    rec_mes=B.receive_order(QA.QA_Event(order=item))
    print(rec_mes)
    AccountT0.receive_deal(rec_mes)

{'header': {'source': 'market', 'status': 200, 'code': '000001', 'session': {'user': None, 'strategy': None, 'account': 'Acc_2JZYw3aV'}, 'order_id': 'Order_eFlLzTK3', 'trade_id': 'Trade_zMKlUVOu'}, 'body': {'order': {'price': 11.01, 'code': '000001', 'amount': 10000, 'date': '2018-05-10', 'datetime': '2018-05-10 15:00:00', 'towards': 1}, 'fee': {'commission': 27.525000000000002, 'tax': 0}}}


In [54]:
AccountT0.cash

[200000, 309807.5, 199679.975]

In [55]:
AccountT0.sell_available

code
000001    0
Name: amount, dtype: int64

In [56]:
AccountT0.hold_available

code
000001    0.0
Name: amount, dtype: float64

In [57]:
AccountT0.hold

code
000001    10000.0
Name: amount, dtype: float64

In [58]:
AccountT0.settle()

In [59]:
AccountT0.daily_hold

,code,000001
date,account_cookie,
2018-05-10,Acc_2JZYw3aV,0.0


In [60]:
AccountT0.daily_cash

,,cash,datetime,date,account_cookie
date,account_cookie,,,,
2018-05-10,Acc_2JZYw3aV,199680,2018-05-10 15:00:00,2018-05-10,Acc_2JZYw3aV


In [61]:
AccountT0.hold_table()

code
000001    10000.0
Name: amount, dtype: float64

In [62]:
AccountT0.hold_price()

code
000001    (0, 0)
dtype: object

In [63]:
AccountT0.datetime

'2018-05-11 09:30:00'

In [64]:
AccountT0.sell_available

code
000001    10000.0
Name: amount, dtype: float64

In [65]:
risk_t0=QA.QA_Risk(AccountT0)

In [66]:
risk_t0.init_assets

{'cash': 200000, 'hold': {'000001': 10000}}

In [67]:
risk_t0.init_cash-risk_t0.assets

date
2018-05-10    320.025
Name: 0, dtype: float64

In [68]:
risk_t0.assets.iloc[-1]/risk_t0.init_cash-1

-0.0016001249999999523

In [69]:
risk_t0.profit_construct

{'total_buyandsell': -99.99,
 'total_tax': -165.0,
 'total_commission': -55.03,
 'total_profit': -320.02}